在沙盒环境，构建机器学习+优化器构建多因子股票组合

In [1]:
import os, copy, datetime
os.environ['ULTRON_DATA'] = 'minden'
import pandas as pd
import numpy as np

In [2]:
from ultron.env import *
from ultron.kdutils.file import load_pickle
from ultron.optimize.model.loader import load_model
from ultron.factor.data.processing import factor_processing
from ultron.factor.data.standardize import standardize
from ultron.factor.data.winsorize import winsorize_normal
from ultron.factor.othgnz.othgnz_engine import OthgnzEngine
from ultron.factor.combine.combine_engine import CombineEngine
from ultron.strategy.stk_strategy import create_params
from ultron.strategy.strategy import StrategyEngine

/var/log/ultron/2022-09-27.log


In [3]:
from ultron.sentry.Analysis.CrossSectionValueHolders import CSPercentileSecurityValueHolder
from ultron.sentry.Analysis.CrossSectionValueHolders import CSRankedSecurityValueHolder
from ultron.sentry.Analysis.SecurityValueHolders import SecurityDeltaValueHolder
from ultron.sentry.Analysis.SecurityValueHolders import SecurityLatestValueHolder
from ultron.sentry.Analysis.SecurityValueHolders import SecurityShiftedValueHolder
from ultron.sentry.Analysis.TechnicalAnalysis.StatelessTechnicalAnalysers import SecurityLogValueHolder
from ultron.sentry.Analysis.TechnicalAnalysis.StatelessTechnicalAnalysers import SecuritySignValueHolder
from ultron.sentry.Analysis.TechnicalAnalysis.StatefulTechnicalAnalysers import SecurityMovingMax
from ultron.sentry.Analysis.TechnicalAnalysis import SecurityMovingCorrelation
from ultron.sentry.Analysis.TechnicalAnalysis import SecurityMovingRank

In [4]:
enable_example_env()

2022-09-27 19:26:38,456 - [env.py:67] - ultron - INFO - enable example env will only read /home/kerry/ultron/rom/sandbox/minden


#### 加载指标数据

In [5]:
indicator_data = pd.read_csv(os.path.join(g_project_data, 'indicator.csv'), index_col=0)
indicator_data['trade_date'] = pd.to_datetime(indicator_data['trade_date'])
indicator_data = indicator_data.rename(columns={'preClosePrice':'pre_close','openPrice':'open',
                        'closePrice':'close','highestPrice':'high','lowestPrice':'low',
                        'turnoverVol':'volume'}).drop(['turnoverValue'],axis=1).set_index(['trade_date'])
indicator_data = indicator_data.sort_values(by=['trade_date','code']).reset_index()
indicator_data.head()

,trade_date,code,pre_close,open,high,low,close,volume,p_change,atr21,atr14,key,date_week
0,2018-10-15,300002,3.070,3.080,3.120,2.990,3.000,9880442,-0.023065,0.140328,0.140576,328,0
1,2018-10-15,300009,8.667,8.736,8.979,8.580,8.742,21076577,0.008616,0.391429,0.388621,329,0
2,2018-10-15,300012,5.363,5.433,5.711,5.433,5.592,22248128,0.041813,0.262680,0.255165,330,0
3,2018-10-15,300014,6.192,6.192,6.465,6.134,6.150,25110741,-0.006806,0.351984,0.369320,331,0
4,2018-10-15,300017,7.362,7.322,7.362,7.004,7.074,46583251,-0.039906,0.342257,0.344513,332,0


#### 加载行业数据

In [6]:
industry_data = pd.read_csv(os.path.join(g_project_data, 'industry_data.csv'), index_col=0)
industry_data['trade_date'] = pd.to_datetime(industry_data['trade_date'])
industry_data.head()

,trade_date,code,industryID1,industryName1
0,2020-08-17,300002,1030325,计算机
1,2020-08-17,300009,1030317,医药生物
2,2020-08-17,300012,1030328,综合
3,2020-08-17,300014,1030308,电气设备
4,2020-08-17,300017,1030327,通信


#### 构建5个alpha191因子作为当前组合因子

In [7]:
def Alpha191_1(data, interval=6):
    exp1 = CSPercentileSecurityValueHolder(
        SecurityDeltaValueHolder(1, SecurityLogValueHolder(SecurityLatestValueHolder('volume'))))
    exp2 = CSPercentileSecurityValueHolder((SecurityLatestValueHolder('close') - \
                SecurityLatestValueHolder('open')) / SecurityLatestValueHolder('open'))

    exp3 = -1 * SecurityMovingCorrelation(interval, exp1, CSRankedSecurityValueHolder(exp2))
    return exp3.transform(data.set_index('trade_date'),name='alpha191_1_' + str(interval) + 'D',
               category_field='code')

def Alpha191_2(data, interval=1):
    exp1 = SecurityLatestValueHolder('close'
                         ) * 2  - SecurityLatestValueHolder('low'
                                                           ) - SecurityLatestValueHolder('high')
    exp2 = SecurityLatestValueHolder('high') - SecurityLatestValueHolder('low')
    exp3 = SecurityDeltaValueHolder(interval, exp1 / exp2) * -1
    return exp3.transform(data.set_index('trade_date'),name='alpha191_2_' + str(interval) + 'D',
               category_field='code')

def Alpha191_5(data, interval=5):
    exp1 = SecurityMovingRank(interval, 'volume')
    exp2 = SecurityMovingRank(interval, 'high')
    exp3 = SecurityMovingCorrelation(interval, exp1 , exp2)
    exp4 = SecurityMovingMax(3, exp3) * -1
    return exp4.transform(data.set_index('trade_date'),name='alpha191_5_' + str(interval) + 'D',
               category_field='code')

def Alpha191_6(data, interval=4):
    exp1 = SecuritySignValueHolder(SecurityDeltaValueHolder(interval,
        SecurityLatestValueHolder('open') * 0.85 + SecurityLatestValueHolder('high') * 0.15))
    
    exp2 = CSPercentileSecurityValueHolder(exp1) * -1
    return exp2.transform(data.set_index('trade_date'),name='alpha191_6_' + str(interval) + 'D',
               category_field='code')


def Alpha191_14(data, interval=1):
    exp1 = SecurityLatestValueHolder('close') - \
SecurityShiftedValueHolder(interval, SecurityLatestValueHolder('close'))
    return exp1.transform(data.set_index('trade_date'),name='alpha191_14_' + str(interval) + 'D',
               category_field='code')

#### 计算因子值

In [8]:
alpha1 = Alpha191_1(indicator_data, interval=6).reset_index().set_index(['trade_date','code'])
alpha2 = Alpha191_2(indicator_data, interval=6).reset_index().set_index(['trade_date','code'])
alpha5 = Alpha191_5(indicator_data, interval=6).reset_index().set_index(['trade_date','code'])
alpha6 = Alpha191_6(indicator_data, interval=6).reset_index().set_index(['trade_date','code'])
alpha14 = Alpha191_14(indicator_data, interval=6).reset_index().set_index(['trade_date','code'])

/home/kerry/work/workenv/ultron/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':
/home/kerry/work/workenv/ultron/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in true_divide
/home/kerry/work/workenv/ultron/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


In [9]:
factors_data = pd.concat([alpha1,alpha2,alpha5,alpha6,alpha14],axis=1).dropna().reset_index()
factors_data.head()

,trade_date,code,alpha191_1_6D,alpha191_2_6D,alpha191_5_6D,alpha191_6_6D,alpha191_14_6D
0,2018-10-23,300002,-0.348808,-0.068376,-0.901504,-0.643939,0.290
1,2018-10-23,300009,0.234440,-0.293772,-0.993808,-0.643939,0.599
2,2018-10-23,300012,-0.439858,-0.186259,-0.947921,-0.643939,0.665
3,2018-10-23,300014,0.452402,-0.268342,-0.911465,-0.643939,0.100
4,2018-10-23,300017,-0.758462,-0.112536,-1.000000,-0.643939,0.635


In [10]:
diff_cols = ['trade_date','code']
features = [col  for col in factors_data.columns if col not in diff_cols]

In [11]:
neutralized_styles = industry_data['industryID1'].unique().tolist()
industry_dummy = pd.get_dummies(industry_data.set_index(['trade_date','code'])['industryID1']).reset_index()
industry_dummy.head()

,trade_date,code,1030303,1030305,1030308,1030309,1030310,1030312,1030314,1030317,1030318,1030324,1030325,1030326,1030327,1030328
0,2020-08-17,300002,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,2020-08-17,300009,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,2020-08-17,300012,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,2020-08-17,300014,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,2020-08-17,300017,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [12]:
def _weighted(data, equal=1):
    data['factor'] =1
    weighted =  1 / len(data)
    weighted = pd.DataFrame([weighted for i in range(0, len(data))],
                                columns=['factor'],
                                index=data.index)
    weighted['code'] = data['code'].values
    weighted = weighted.reset_index().drop(['trade_date'],axis=1)
    return weighted.rename(columns={'factor':'weight'})
benchmark_weighed = factors_data[['trade_date','code']].set_index('trade_date').groupby(
    level=['trade_date']).apply(
    lambda x: _weighted(x)).reset_index().drop(['level_1'],axis=1)
benchmark_weighed.head()

,trade_date,weight,code
0,2018-10-23,0.015625,300002
1,2018-10-23,0.015625,300009
2,2018-10-23,0.015625,300012
3,2018-10-23,0.015625,300014
4,2018-10-23,0.015625,300017


In [13]:
factors_data = factors_data.merge(industry_dummy,on=['trade_date','code']).merge(
    industry_data,on=['trade_date','code']).merge(
    indicator_data[['trade_date','code','close']],on=['trade_date','code']).merge(
    benchmark_weighed,on=['trade_date','code']).drop(['industryName1'],axis=1).rename(
    columns={'industryID1':'industry_code','close':'closePrice'})
factors_data.head()

,trade_date,code,alpha191_1_6D,alpha191_2_6D,alpha191_5_6D,alpha191_6_6D,alpha191_14_6D,1030303,1030305,1030308,...,1030317,1030318,1030324,1030325,1030326,1030327,1030328,industry_code,closePrice,weight
0,2019-12-31,300002,0.358900,-0.914286,-0.488901,-0.287879,0.020,0,0,0,...,0,0,0,1,0,0,0,1030325,3.260,0.015152
1,2019-12-31,300009,-0.721964,-0.851203,-0.772881,-0.287879,0.056,0,0,0,...,1,0,0,0,0,0,0,1030317,9.506,0.015152
2,2019-12-31,300012,0.271081,-0.927395,-0.433861,-0.787879,0.399,0,0,0,...,0,0,0,0,0,0,1,1030328,14.866,0.015152
3,2019-12-31,300014,0.323163,-0.360846,-0.119523,-0.287879,-1.490,0,0,0,...,0,0,0,0,0,0,0,1030312,26.321,0.015152
4,2019-12-31,300017,0.382712,-1.144134,-0.745356,-0.287879,0.199,0,0,0,...,0,0,0,0,0,1,0,1030327,9.477,0.015152


In [14]:
effective_codes = factors_data.industry_code.unique().tolist() ### 有效行业
invalid_codes = [] ### 无效行业
turn_over_target = 0.8  ### 换手率
lbound = 0. ### 单个标的优化下限
ubound = 0.1 ### 单个标的优化上限
benchmark_lower = 0.9 ## 基于基准下限
benchmark_upper = 1.001 ## 基于基准上限
total_lower = 0. ## 组合下限
total_upper = 1.01 ## 组合上限
effective_industry_lower = -0.1 ## 有效行业下限
effective_industry_upper = 0.1 ## 有效行业上限
invalid_industry_lower = -0.1 ## 无效行业下限
invalid_industry_upper = 0.1 ## 无效行业上限
method = 'long_risk_neutral'
synthetize_opts = {'method':'equal'}

In [15]:
params = create_params(
    effective_codes=effective_codes, invalid_codes=invalid_codes,
    turn_over_target=turn_over_target, 
    lbound=lbound, ubound=ubound,
    benchmark_lower=benchmark_lower,benchmark_upper=benchmark_upper,
    total_lower=total_lower,total_upper=total_upper,
    effective_industry_lower=effective_industry_lower,
    effective_industry_upper=effective_industry_upper,
    method=method,synthetize_opts=synthetize_opts,
    is_benchmark=1)

#### 加载模型

In [16]:
desc = load_pickle(os.path.join(g_project_data, 'model.h5'))
new_model = load_model(desc)

please wait! load_pickle....: /home/kerry/ultron/rom/sandbox/minden/model.h5


In [17]:
models = {}
factors_data_groups = factors_data.groupby('trade_date')
for ref_date, _ in factors_data_groups:
    models[ref_date] = new_model

In [18]:
strategy = StrategyEngine.create_class(name='stk')(alpha_model=models, total_data=factors_data, features=features,
                    start_date=datetime.datetime(2018,1,4))

In [19]:
match_df, ret_df, positions = strategy.run(params=params)

2022-09-27 19:26:50,101 - [stk_strategy.py:477] - ultron - INFO - starting standardize data ...
2022-09-27 19:26:50,102 - [stk_strategy.py:478] - ultron - INFO - running setting finished ...
2022-09-27 19:26:50,106 - [stk_strategy.py:483] - ultron - INFO - starting industry median data ...
2022-09-27 19:26:50,246 - [stk_strategy.py:496] - ultron - INFO - starting  factor processing data ...
2022-09-27 19:26:53,914 - [stk_strategy.py:515] - ultron - INFO - running setting finished ...
2022-09-27 19:26:54,127 - [stk_strategy.py:522] - ultron - INFO - returns finished ...
2022-09-27 19:26:54,152 - [stk_strategy.py:541] - ultron - INFO - running setting finished ...
2022-09-27 19:26:54,152 - [stk_strategy.py:340] - ultron - INFO - synthetize method:equal
2022-09-27 19:26:54,244 - [stk_strategy.py:553] - ultron - INFO - starting re-balance ...
2022-09-27 19:26:54,258 - [stk_strategy.py:570] - ultron - INFO - running setting finished ...
2022-09-27 19:26:54,483 - [stk_strategy.py:619] - ultr

2022-09-27 19:26:56,106 - [stk_strategy.py:619] - ultron - INFO - 2020-04-15 00:00:00 re-balance: 65 codes
2022-09-27 19:26:56,130 - [stk_strategy.py:619] - ultron - INFO - 2020-04-16 00:00:00 re-balance: 64 codes
2022-09-27 19:26:56,154 - [stk_strategy.py:619] - ultron - INFO - 2020-04-17 00:00:00 re-balance: 63 codes
2022-09-27 19:26:56,178 - [stk_strategy.py:619] - ultron - INFO - 2020-04-20 00:00:00 re-balance: 65 codes
2022-09-27 19:26:56,202 - [stk_strategy.py:619] - ultron - INFO - 2020-04-21 00:00:00 re-balance: 66 codes
2022-09-27 19:26:56,226 - [stk_strategy.py:619] - ultron - INFO - 2020-04-22 00:00:00 re-balance: 66 codes
2022-09-27 19:26:56,250 - [stk_strategy.py:619] - ultron - INFO - 2020-04-23 00:00:00 re-balance: 65 codes
2022-09-27 19:26:56,274 - [stk_strategy.py:619] - ultron - INFO - 2020-04-24 00:00:00 re-balance: 64 codes
2022-09-27 19:26:56,297 - [stk_strategy.py:619] - ultron - INFO - 2020-04-27 00:00:00 re-balance: 64 codes
2022-09-27 19:26:56,322 - [stk_strate

2022-09-27 19:26:57,955 - [stk_strategy.py:619] - ultron - INFO - 2020-08-07 00:00:00 re-balance: 66 codes
2022-09-27 19:26:57,978 - [stk_strategy.py:619] - ultron - INFO - 2020-08-10 00:00:00 re-balance: 66 codes
2022-09-27 19:26:58,002 - [stk_strategy.py:619] - ultron - INFO - 2020-08-11 00:00:00 re-balance: 66 codes
2022-09-27 19:26:58,025 - [stk_strategy.py:619] - ultron - INFO - 2020-08-12 00:00:00 re-balance: 66 codes
2022-09-27 19:26:58,049 - [stk_strategy.py:619] - ultron - INFO - 2020-08-13 00:00:00 re-balance: 66 codes
2022-09-27 19:26:58,073 - [stk_strategy.py:619] - ultron - INFO - 2020-08-14 00:00:00 re-balance: 66 codes
2022-09-27 19:26:58,096 - [stk_strategy.py:619] - ultron - INFO - 2020-08-17 00:00:00 re-balance: 66 codes
2022-09-27 19:26:58,120 - [stk_strategy.py:619] - ultron - INFO - 2020-08-18 00:00:00 re-balance: 66 codes
2022-09-27 19:26:58,144 - [stk_strategy.py:619] - ultron - INFO - 2020-08-19 00:00:00 re-balance: 66 codes
2022-09-27 19:26:58,171 - [stk_strate

2022-09-27 19:26:59,931 - [stk_strategy.py:619] - ultron - INFO - 2020-12-02 00:00:00 re-balance: 66 codes
2022-09-27 19:26:59,955 - [stk_strategy.py:619] - ultron - INFO - 2020-12-03 00:00:00 re-balance: 66 codes
2022-09-27 19:26:59,978 - [stk_strategy.py:619] - ultron - INFO - 2020-12-04 00:00:00 re-balance: 66 codes
2022-09-27 19:27:00,002 - [stk_strategy.py:619] - ultron - INFO - 2020-12-07 00:00:00 re-balance: 66 codes
2022-09-27 19:27:00,026 - [stk_strategy.py:619] - ultron - INFO - 2020-12-08 00:00:00 re-balance: 66 codes
2022-09-27 19:27:00,050 - [stk_strategy.py:619] - ultron - INFO - 2020-12-09 00:00:00 re-balance: 66 codes
2022-09-27 19:27:00,074 - [stk_strategy.py:619] - ultron - INFO - 2020-12-10 00:00:00 re-balance: 66 codes
2022-09-27 19:27:00,098 - [stk_strategy.py:619] - ultron - INFO - 2020-12-11 00:00:00 re-balance: 66 codes
2022-09-27 19:27:00,122 - [stk_strategy.py:619] - ultron - INFO - 2020-12-14 00:00:00 re-balance: 66 codes
2022-09-27 19:27:00,148 - [stk_strate

2022-09-27 19:27:01,811 - [stk_strategy.py:619] - ultron - INFO - 2021-03-29 00:00:00 re-balance: 66 codes
2022-09-27 19:27:01,836 - [stk_strategy.py:619] - ultron - INFO - 2021-03-30 00:00:00 re-balance: 66 codes
2022-09-27 19:27:01,860 - [stk_strategy.py:619] - ultron - INFO - 2021-03-31 00:00:00 re-balance: 66 codes
2022-09-27 19:27:01,885 - [stk_strategy.py:619] - ultron - INFO - 2021-04-01 00:00:00 re-balance: 66 codes
2022-09-27 19:27:01,909 - [stk_strategy.py:619] - ultron - INFO - 2021-04-02 00:00:00 re-balance: 66 codes
2022-09-27 19:27:01,933 - [stk_strategy.py:619] - ultron - INFO - 2021-04-06 00:00:00 re-balance: 66 codes
2022-09-27 19:27:01,959 - [stk_strategy.py:619] - ultron - INFO - 2021-04-07 00:00:00 re-balance: 66 codes
2022-09-27 19:27:01,983 - [stk_strategy.py:619] - ultron - INFO - 2021-04-08 00:00:00 re-balance: 66 codes
2022-09-27 19:27:02,007 - [stk_strategy.py:619] - ultron - INFO - 2021-04-09 00:00:00 re-balance: 66 codes
2022-09-27 19:27:02,030 - [stk_strate

...

2022-09-27 19:27:11,919 - [stk_strategy.py:687] - ultron - INFO - 2021-06-09 00:00:00: turn over 0.8, returns 0.0119, benchmark returns 0.0175
2022-09-27 19:27:11,928 - [stk_strategy.py:687] - ultron - INFO - 2021-06-10 00:00:00: turn over 0.8, returns -0.0019, benchmark returns -0.008
2022-09-27 19:27:11,934 - [stk_strategy.py:687] - ultron - INFO - 2021-06-11 00:00:00: turn over 0.8, returns -0.0161, benchmark returns -0.0015
2022-09-27 19:27:11,941 - [stk_strategy.py:687] - ultron - INFO - 2021-06-15 00:00:00: turn over 0.8, returns -0.0079, benchmark returns -0.017
2022-09-27 19:27:11,948 - [stk_strategy.py:687] - ultron - INFO - 2021-06-16 00:00:00: turn over 0.8, returns 0.0094, benchmark returns 0.0103
2022-09-27 19:27:11,955 - [stk_strategy.py:687] - ultron - INFO - 2021-06-17 00:00:00: turn over 0.697, returns 0.007, benchmark returns 0.0099
2022-09-27 19:27:11,962 - [stk_strategy.py:687] - ultron - INFO - 2021-06-18 00:00:00: turn over 0.8, returns 0.0012, benchmark returns 0

2022-09-27 19:27:12,313 - [stk_strategy.py:687] - ultron - INFO - 2021-08-31 00:00:00: turn over 0.8, returns 0.008, benchmark returns -0.0038
2022-09-27 19:27:12,319 - [stk_strategy.py:687] - ultron - INFO - 2021-09-01 00:00:00: turn over 0.8, returns -0.0144, benchmark returns -0.006
2022-09-27 19:27:12,326 - [stk_strategy.py:687] - ultron - INFO - 2021-09-02 00:00:00: turn over 0.8, returns -0.0064, benchmark returns 0.0015
2022-09-27 19:27:12,332 - [stk_strategy.py:687] - ultron - INFO - 2021-09-03 00:00:00: turn over 0.6, returns 0.0233, benchmark returns 0.0237
2022-09-27 19:27:12,339 - [stk_strategy.py:687] - ultron - INFO - 2021-09-06 00:00:00: turn over 0.8, returns 0.0159, benchmark returns 0.0116
2022-09-27 19:27:12,346 - [stk_strategy.py:687] - ultron - INFO - 2021-09-07 00:00:00: turn over 0.6788, returns 0.0246, benchmark returns 0.0204
2022-09-27 19:27:12,353 - [stk_strategy.py:687] - ultron - INFO - 2021-09-08 00:00:00: turn over 0.8, returns 0.0003, benchmark returns -

2022-09-27 19:27:12,716 - [stk_strategy.py:687] - ultron - INFO - 2021-11-29 00:00:00: turn over 0.8, returns 0.0114, benchmark returns 0.0035
2022-09-27 19:27:12,723 - [stk_strategy.py:687] - ultron - INFO - 2021-11-30 00:00:00: turn over 0.8, returns 0.0033, benchmark returns 0.0073
2022-09-27 19:27:12,729 - [stk_strategy.py:687] - ultron - INFO - 2021-12-01 00:00:00: turn over 0.8, returns -0.0282, benchmark returns -0.017
2022-09-27 19:27:12,736 - [stk_strategy.py:687] - ultron - INFO - 2021-12-02 00:00:00: turn over 0.8, returns -0.0015, benchmark returns 0.0005
2022-09-27 19:27:12,743 - [stk_strategy.py:687] - ultron - INFO - 2021-12-03 00:00:00: turn over 0.8, returns -0.0113, benchmark returns -0.0163
2022-09-27 19:27:12,750 - [stk_strategy.py:687] - ultron - INFO - 2021-12-06 00:00:00: turn over 0.8, returns 0.0214, benchmark returns 0.0027
2022-09-27 19:27:12,757 - [stk_strategy.py:687] - ultron - INFO - 2021-12-07 00:00:00: turn over 0.8, returns 0.0163, benchmark returns 0.

2022-09-27 19:27:13,111 - [stk_strategy.py:687] - ultron - INFO - 2022-02-24 00:00:00: turn over 0.8, returns 0.0232, benchmark returns 0.0188
2022-09-27 19:27:13,118 - [stk_strategy.py:687] - ultron - INFO - 2022-02-25 00:00:00: turn over 0.8, returns 0.0092, benchmark returns 0.0097
2022-09-27 19:27:13,124 - [stk_strategy.py:687] - ultron - INFO - 2022-02-28 00:00:00: turn over 0.8, returns -0.0043, benchmark returns -0.0021
2022-09-27 19:27:13,131 - [stk_strategy.py:687] - ultron - INFO - 2022-03-01 00:00:00: turn over 0.8, returns -0.0044, benchmark returns -0.004
2022-09-27 19:27:13,138 - [stk_strategy.py:687] - ultron - INFO - 2022-03-02 00:00:00: turn over 0.8, returns -0.0134, benchmark returns -0.0108
2022-09-27 19:27:13,145 - [stk_strategy.py:687] - ultron - INFO - 2022-03-03 00:00:00: turn over 0.8, returns 0.0017, benchmark returns -0.0078
2022-09-27 19:27:13,152 - [stk_strategy.py:687] - ultron - INFO - 2022-03-04 00:00:00: turn over 0.8, returns -0.0612, benchmark returns

2022-09-27 19:27:13,510 - [stk_strategy.py:687] - ultron - INFO - 2022-05-24 00:00:00: turn over 0.8, returns 0.019, benchmark returns 0.0171
2022-09-27 19:27:13,516 - [stk_strategy.py:687] - ultron - INFO - 2022-05-25 00:00:00: turn over 0.8, returns 0.0054, benchmark returns 0.0056
2022-09-27 19:27:13,523 - [stk_strategy.py:687] - ultron - INFO - 2022-05-26 00:00:00: turn over 0.8, returns -0.0034, benchmark returns -0.0073
2022-09-27 19:27:13,530 - [stk_strategy.py:687] - ultron - INFO - 2022-05-27 00:00:00: turn over 0.8, returns 0.016, benchmark returns 0.0104
2022-09-27 19:27:13,537 - [stk_strategy.py:687] - ultron - INFO - 2022-05-30 00:00:00: turn over 0.8, returns 0.0187, benchmark returns 0.0212
2022-09-27 19:27:13,544 - [stk_strategy.py:687] - ultron - INFO - 2022-05-31 00:00:00: turn over 0.8, returns 0.0136, benchmark returns 0.0012
2022-09-27 19:27:13,551 - [stk_strategy.py:687] - ultron - INFO - 2022-06-01 00:00:00: turn over 0.8, returns 0.0063, benchmark returns 0.0081